#### Import libraries 

In [1]:
import numpy as np 
import pandas as pd 
import xarray as xr
import matplotlib.pyplot as plt 
import time 

#### Paths used

In [2]:
path = '/home/marcos/Documentos/6.-Post_Doctorado_CIDE_Deep_Learning/Proyectos/Monitor_Vientos/2.-Viento_Medio/Preprocessing_Data/'
path_aemet = '3.-data_with_NAN_values_corrected/'
nc_mask_aemet = 'mask_aemet_daily-wind_1961-2021_010x010_144.nc'
path_out = '2.-Mask_creation/Stations_removed_randomly_ratio/'

#### Definition of the function 

In [3]:
def random_station_drop(ds, ratio):
    #output dataframe
    df = None

    #Now, it expresses the deactivated ratio of AEMET stations
    ratio_value = 1 - ratio

    #ds.isel(time=-126)["mask"].plot()

    #Take just 1 time and forget the time dimension. I only want to deactivate stations in the XY dimension
    coords_mask = ds.isel(time=-1).to_dataframe().drop(columns ="time")#reset_index()

    daout = coords_mask.where(coords_mask.isnull(), 0)

    aemet_stations = coords_mask.loc[coords_mask["mask"]>0]

    shuffle_index = np.random.permutation(len(aemet_stations))

    est_index_down = shuffle_index[:int(len(aemet_stations) * ratio_value)]

    mod_ae_stat = aemet_stations.copy()
    mod_ae_stat["mask"] = 1
    mod_ae_stat["mask"].iloc[est_index_down] = 0

    mod_ae_stat = mod_ae_stat.loc[mod_ae_stat["mask"] == 1]

    if len(mod_ae_stat)>0:
        daout.loc[mod_ae_stat.index] = 1
        df = daout 

    dsmask = df.to_xarray()
    newmask = ds * dsmask 

    return newmask

#### Main function

In [4]:
if __name__ == "__main__":

    dsmask = xr.open_dataset(path + path_aemet + nc_mask_aemet)

    dsmask = dsmask.sel(time=slice("2010-01-01", "2021-12-31"))

    #ratios = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    ratios = [0.3]

    for ratio in ratios:
        t1 = time.time()

        ds1 = random_station_drop(dsmask, ratio)

        ds1 = ds1.transpose("time", "latitude", "longitude")

        t2 = time.time()
        
        fout = "mask_aemet_daily-wind_2010-2021_144_" + str(ratio).replace(".", "p") + "_random-stations.nc"

        ds1.to_netcdf(path_out + fout)

        t3 = time.time()

        print("ratio: ", ratio)
        
        print()
        print("t1: ", t1)
        print("t2: ", t2)
        print("t3: ", t3)

        print()
        print("D_t_21: ", t2 - t1)
        print("D_t_32: ", t3 - t2)

        print()


ratio:  0.3

t1:  1705941520.6195667
t2:  1705941520.9215074
t3:  1705941521.4267673

D_t_21:  0.3019406795501709
D_t_32:  0.5052599906921387

